In [1]:
from mrjob.job import MRJob
import psycopg2

In [ ]:
class AggregateProduct(MRJob):

    def reducer_init(self):
        self.conn= psycopg2.connect(database= 'database', user ='postgres', password='mekanika', host='localhost', port='5432')

    def mapper(self, _, line):
        item=line.strip().split(',')
        year =item[1][-4:]
        yield item[1], int(item[4])

    def reducer(self, key, value):
        yield key, sum(values)

    def store(self,key,values):
        self.cur=self.conn.cursor()
        self.cur.execute("insert into total_order_yearly (year, total_order) values(%s, %s)",(key,values))

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper,
                reducer=self.reducer
            )
            MRStep(
                mapper=self.mapper,
                reducer_init=self.reducer_init,
                reducer=self.reducer
            )
        ]
    def mapper_final(self):
        self.conn.commit()
        self.conn.close()

if __name__ == '__main__':
    AggregateProduct.run()